In [1]:
import ibis
ibis.options.interactive = True
from ibis import _

In [3]:

df_8_35 = (
    ibis.read_parquet('data/flight-server-arrow-8-grpc-1-35.parquet')
    .mutate(arrow_version="8.0.0", grpc_version="1.35.0")
    .mutate(start_time = _.start_time - _.start_time.min())
)
df_9_35 = (
    ibis.read_parquet('data/flight-server-arrow-9-grpc-1-35.parquet')
    .mutate(arrow_version="9.0.0", grpc_version="1.35.0")
    .mutate(start_time = _.start_time - _.start_time.min())
)
df_9_46 = (
    ibis.read_parquet('data/flight-server-arrow-9-grpc-1-46.parquet')
    .mutate(arrow_version="9.0.0", grpc_version="1.46.0")
    .mutate(start_time = _.start_time - _.start_time.min())
)

df = df_8_35.union(df_9_35).union(df_9_46)

In [4]:
aggregated_timings = (
    df
    .mutate(interval=df.start_time.round().cast('int64'))
    .group_by(['arrow_version', 'grpc_version', 'interval'])
    .aggregate(
        count=_.count(),
        max_latency=_.latency.max(),
        min_latency=_.latency.min(),
        mean_latency=_.latency.mean(),
    )
    .order_by('interval')
    .mutate(version_label = "Arrow " + _.arrow_version + " / gRPC " + _.grpc_version)
)

In [5]:
import altair as alt
import vegafusion as vf
vf.enable()

vegafusion.enable(mimetype='html', row_limit=10000, embed_options=None)

In [11]:
base = alt.Chart(aggregated_timings.to_pandas()).mark_circle().encode(
    x='interval',
    y='max_latency',
    color='version_label',
)

base + base.transform_loess('interval', 'max_latency', groupby=['version_label']).mark_line(size=4)

alt.LayerChart(...)

In [13]:
base = alt.Chart(aggregated_timings.to_pandas()).mark_circle().encode(
    x='interval',
    y='min_latency',
    color='version_label',
)

base + base.transform_loess('interval', 'min_latency', groupby=['version_label']).mark_line(size=4)

alt.LayerChart(...)